## NHANES Project

Doug Johnson document

We're trying to replicate and update Hankinson

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nhanes.household_adult_file import HouseholdAdultFile
import nhanes_loader
import importlib
importlib.reload(nhanes_loader)
import nhanes.household_adult_file
importlib.reload(nhanes.household_adult_file)


## Import exam file

In [ ]:
from nhanes.exam_file import ExamFile

In [ ]:
exam = ExamFile()

In [ ]:
exam.data.shape

In [ ]:
exam.cols

In [ ]:
exam.data.DMARACER

In [ ]:
exam.data.DMARACER.value_counts()

## Import Household Youth File

In [ ]:
from nhanes.household_youth_file import HouseholdYouthFile

In [ ]:
youth = HouseholdYouthFile()

In [ ]:
youth.cols

## Import Household Adult File

In [ ]:
nh = HouseholdAdultFile()
nh.data

## Combine adult and exam files

match files on the SEQN field

In [ ]:
adult_exam = nh.data.set_index('SEQN').join(exam.data.set_index('SEQN'),rsuffix='_exam')

## Hankinson Exclusion

Ages - have none over 89; 17-89

In [ ]:
adult_exam.HSAGEIR.value_counts()

Check for age of 16

In [205]:
adult_exam.query('HSAGEIR < 17').shape

(0, 3604)

Any age  .shape - returns row and column count of a dataset

.describe() - returns statistics about numerical columns

.dtypes - returns the data type of each column

.count() - number of total values in column

.value_counts() - returns object counting counts of unique values

In [189]:
adult_exam.query('HSAGEIR>0').shape

(20050, 3604)

Check have Height

In [184]:
adult_exam.query('BMPHT').shape

(20050, 3604)

describe SPPMANEU

Any spirometry  H 16484 

In [209]:
adult_exam.SPPMANEU.value_counts()

5.0     7569
6.0     2553
4.0     1932
7.0     1544
88.0    1310
8.0     1238
3.0      668
9.0      400
2.0      292
10.0     233
1.0      226
0.0      144
11.0      23
13.0       8
12.0       8
14.0       1
16.0       1
Name: SPPMANEU, dtype: int64

SPPMANEU: successful spirometries -should=16840-370=16470
    

adult_exam.SPPMANEU.value_counts()

any spirometry (in adult age >=17  H 16484  not=88

In [219]:
adult_exam.query('SPPMANEU != 88').shape

(18740, 3604)

In [217]:
adult_exam.query('SPPMANEU < 88').shape

(16840, 3604)

In [220]:
adult_exam.query('SPPMANEU > -1').shape

(18150, 3604)

number 2 or more tests  H 277, 16207  we have 16470 (+263)

In [127]:
adult_exam.query('2 <= SPPMANEU < 88').shape

(16470, 3604)

and age < 90 H 68,16139  we have 84  16386 (+26, +247)

In [224]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR >89').shape

(84, 3604)

In [147]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').shape

(16386, 3604)

adult_exam.query('HSAGEIR >= 90').shape

and race/ethnicity not other  H 636, 15503

adult_exam.query('HSAGEIR > 0').shape

and race/ethnicity as other  H 636  15503  we have 639, 15747 (+3, +244)

In [228]:
adult_exam.DMARETHN.value_counts()

1    8483
2    5486
3    5306
4     775
Name: DMARETHN, dtype: int64

In [226]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN == 4').shape

(639, 3604)

In [227]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').shape

(15747, 3604)

and cigarette smoker H 7,667  7,836  We have  7787 7960 (+120  +124)


In [235]:
adult_exam.HAR1.value_counts()

2    10235
1     9799
8       16
Name: HAR1, dtype: int64

or statement not work, so need HAR1 = 2

In [242]:
adult_exam.query('HAR1 == 1' or 'HAR1 == 8').shape

(9799, 3604)

In [243]:
adult_exam.query('HAR1 == 2').shape

(10235, 3604)

In [245]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 ==2').shape

(7960, 3604)

and cigar or pipe smoker  H 313, 7523  for cigar we have 284, 7676 

cigar HAR23  2 is not cigar

In [150]:
adult_exam.HAR23.value_counts()

2    17951
1     2073
8       21
9        5
Name: HAR23, dtype: int64

In [275]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 ==2').query('HAR23 == 2').shape

(7676, 3604)

and pipe smoker with above H 313 7523  We have 326 7634 (+13 +111)

In [273]:
adult_exam.HAR26.value_counts()

2    18411
1     1609
8       26
9        4
Name: HAR26, dtype: int64

In [276]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 == 2').query('HAR23 == 2').query('HAR26 == 2').shape

(7634, 3604)

and smoked cigarettes/cigar/pipes in 5 d prior to exam  H 408 7115  BUT this question only on Youth.  Adult has question SPPQ3 smoked, heavy meal, meds to help breathing past hour; also SPPQ4 past few day cough/cold/other acute illness; also SPPQ5 past 3 weeks respiratory infection (so want these = 2)

In [ ]:
We have 5965 1669 when exclude all 3

In [277]:
adult_exam.SPPQ3.value_counts()

2.0    16565
8.0     1320
1.0      265
Name: SPPQ3, dtype: int64

In [ ]:
exclude just Q3 - we have 7406 228 lower

In [281]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 == 2').query('HAR23 == 2').query('HAR26 == 2').query('SPPQ3 == 2').shape

(7406, 3604)

In [278]:
adult_exam.SPPQ4.value_counts()

2.0    13557
1.0     3273
8.0     1320
Name: SPPQ4, dtype: int64

exclude Q3 and Q4

In [282]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 == 2').query('HAR23 == 2').query('HAR26 == 2').query('SPPQ3 == 2').query('SPPQ4 == 2').shape

(6120, 3604)

We have 6120 1286 for exclude Q3 and Q4

In [279]:
adult_exam.SPPQ5.value_counts()

2.0    15970
8.0     1320
1.0      860
Name: SPPQ5, dtype: int64

In [280]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 == 2').query('HAR23 == 2').query('HAR26 == 2').query('SPPQ3 == 2').query('SPPQ4 == 2').query('SPPQ5 == 2').shape

(5965, 3604)

and asthma H  454  6661

In [260]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 == 2').query('HAR23 > 1').query('HAR26 > 1').query('HAC1E > 1').shape

(7163, 3604)

and chronic bronchitis H  181, 6480

In [259]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 == 2').query('HAR23 > 1').query('HAR26 > 1').query('HAC1E > 1').query('HAC1F > 1').shape

(6967, 3604)

and emphysema  H 15, 6465

In [258]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARACER < 4').query('HAR1 == 2').query('HAR23 > 1').query('HAR26 > 1').query('HAC1E > 1').query('HAC1F > 1').query('HAC1G > 1').shape

(7314, 3604)

and cancer H 0 6465

In [261]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARACER < 4').query('HAR1 == 2').query('HAR23 > 1').query('HAR1 < 888').query('HAR26 > 1').query('HAC1E > 1').query('HAC1F > 1').query('HAC1G > 1').query('HAC1O > 1').shape

(7119, 3604)

and wheezing 6 month  H 419 6046

In [262]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 == 2').query('HAR23 > 1').query('HAR26 > 1').query('HAC1E > 1').query('HAC1F > 1').query('HAC1G > 1').query('HAC1O > 1').query('HAL6 > 1').shape

(6328, 3604)

and whistling or wheezing  H 112 5934

In [264]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 == 2').query('HAR23 > 1').query('HAR26 > 1').query('HAC1E > 1').query('HAC1F > 1').query('HAC1G > 1').query('HAC1O > 1').query('HAL6 > 1').query('HAL10 > 1').shape

(6204, 3604)

and persistent cough H 158, 5776

In [269]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 > 1').query('HAR23 > 1').query('HAR26 > 1').query('HAC1E > 1').query('HAC1F > 1').query('HAC1G > 1').query('HAC1O > 1').query('HAL6 > 1').query('HAL10 > 1').query('HAL1 > 1').shape

(6045, 3604)

and persistent phlegm H 125  5776

In [268]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 == 2').query('HAR23 > 1').query('HAR26 > 1').query('HAC1E > 1').query('HAC1F > 1').query('HAC1G > 1').query('HAC1O > 1').query('HAL6 > 1').query('HAL10 > 1').query('HAL1 > 1').query('HAL3 > 1').shape

(5918, 3604)

and moderate shortness of breath H  848  4803

In [267]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 == 2').query('HAR23 > 1').query('HAR26 > 1').query('HAC1E > 1').query('HAC1F > 1').query('HAC1G > 1').query('HAC1O > 1').query('HAL6 > 1').query('HAL10 > 1').query('HAL1 > 1').query('HAL3 > 1').query('HAL5 > 1').shape

(5069, 3604)

and age > 80  H  139 4634

In [266]:
adult_exam.query('2 <= SPPMANEU < 88').query('HSAGEIR < 90').query('DMARETHN < 4').query('HAR1 == 2').query('HAR23 > 1').query('HAR26 > 1').query('HAC1E > 1').query('HAC1F > 1').query('HAC1G > 1').query('HAC1O > 1').query('HAL6 > 1').query('HAL10 > 1').query('HAL1 > 1').query('HAL3 > 1').query('HAL5 > 1').query('HSAGEIR < 81').shape

(4883, 3604)

## FINAL # above - H 4,634

Try Xvariables criteria

In [ ]:
exam.data['XSPPMANEU'] = adult_exam.query('2 <= SPPMANEU < 88')

In [126]:
exam.data['XSPPMANEU'].count()

8132

In [ ]:
exam.data['XSPPMANEU'].value_counts()

.describe() returns statistics about numerical columns

In [ ]:
exam.data['XSPPMANEU'].describe()

In [ ]:
exam.data['XSPPMANEU'].describe(include="all")

In [130]:
exam.data['XHSAGEIR90'] = adult_exam.query('HSAGEIR >= 90')

In [139]:
exam.data['XHSAGEIR90'].count()

104

exam.data['XHSMANAGE90'] = any ['SPPMANEU <2', 'SPPMANEU =88','HSAGEIR > 90']  NOT WORK

In [136]:
exam.data['XHSMANAGE90'].count()

31311

exam.data['XSPPMANEU'].unique()

describes apply function to dataset to transform it - then append to DataFrame in a new column

The data frame has object geography

def mapgeography(x):
  If x =="City"
    return 1
  else:
    return 0
    
df['geography_mapped_value') = df.geopgraphy_type.apply(mapgeography)

then df.geography.mapped_value.value_counts()
  to get list of values
  
Could use this method to append values of exclusion criteria, such as

def map1(x):
  If x == 1
    return 1
  else:
    return 0
    
then df['XHAR1'] = df.HAR1.apply(map1)

or for maplt2

def maplt2(x):
  If x <2
    return 1
  else:  
    return 0
    
then df['XSPPMANEU'] = df.SPPMANEU.apply(maplt2)    
    
    

In [ ]:
number = 7

In [ ]:
print (number)

In [ ]:
print ('SPPMANEU < 2')

## Exclusion Youth

youth_exam.query('HSAGEIR < 17').shape

## Reviewing adult file

In [ ]:
cols = nh.cols
cols

In [ ]:
cols[cols.description.str.contains('(?:smok|cig)')]

In [ ]:
nh.hist_amount('HAR4S');

In [ ]:
nh.hist_amount('HSAITMOR',outliers=1200);

In [ ]:
nh.hist_amount('DMARACER', outliers=25);

In [ ]:
cols[cols.description.str.contains('weight|pounds') & ~cols.description.str.contains('(sample|interview|weighting|exam)')]

In [ ]:
exam.data.HAR1.value_counts();

Test Markdown

In [ ]:
nh.hist_amount('HAR4S');

In [ ]:
nh.hist_amount('HSAITMOR',outliers=1200);

In [ ]:
nh.hist_amount('DMARACER', outliers=25);

In [ ]:
cols[cols.description.str.contains('weight|pounds') & ~cols.description.str.contains('(sample|interview|weighting|exam)')]

In [ ]:
exclude = { 'SPPQ1' : 1, 'SPPQ2' : 1, 'SPPQ3' : 1, 'SPPQ4' : 1, 'SPPQ5' : 1}

In [ ]:
exam.data['PEFROVERFVC'] = exam.data.SPPPEAK / exam.data.SPPFVC

In [ ]:
exam.data.PEFROVERFVC.hist(bins='auto', range=(0,5))

In [ ]:
exam.data[(exam.data.HSSEX == 2) & (exam.data.DMARACER == 1)].PEFROVERFVC.hist(bins='auto')